In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
import requests

from tqdm import tqdm

from scorer import *

In [2]:
def send_qwen(prompt):
    r = requests.post('http://10.11.12.131:8080', json=prompt)
    if r.status_code == 200:
        res = r.json()
        print(res)
        if 'result' in res:
            return res['result']
    return ""

In [22]:
send_qwen([
    {'role':'system', 'content':"""You are assistant for analysing model hallucinations. Your task is to extract fragments from model output, containing factually incorrect answers. You need to extract factually incorrect or inconsistent with input fragments from model output."""},
    {'role':'user', 'content':"""

"""}
])

{'error': 'Wrong answer'}


''

In [3]:
def summarise(docs, model_inputs, model_outputs):
    res = []
    for i in tqdm(docs.index):
        prompt = 'You are data analysis assistant, your task is to summarise wiki page according to model output. You will be supplied with model input, model output and relevant wiki page, your task is to write key fact from wiki to verify model output. Write only facts from wiki page.\nmodel output:\n{model_output}\nmodel input:\n{model_input}\nwiki page:\n{txt}'
        res += [send_qwen(prompt.format(model_input=model_inputs[i], model_output=model_outputs[i], txt=docs[i]))]
    return res

In [4]:
df = pd.read_json('/home/admin/projects/semeval2025/semeval_val_RAG.json')
df = df[df['lang']=='EN']

In [5]:
for i in range(1, 5+1):
    df[f'doc_{i}'] = summarise(df[f'doc{i}'], df[f'model_input'], df[f'model_output_text'])

100%|██████████| 50/50 [02:35<00:00,  3.11s/it]


In [7]:
df.head()

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens,doc1,doc2,doc3,doc4,doc5,doc_1,doc_2,doc_3,doc_4,doc_5
100,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]","[-5.5669536591, -11.90533638, -13.0743436813, ...","[Pet, ra, Ġvan, ĠSto, ve, ren, Ġwon, Ġa, Ġsilv...","{'id': 10005036696, 'score': 0.867261899999999...","{'id': 10040058894, 'score': 0.8276253, 'text'...","{'id': 10000352674, 'score': 0.822938899999999...","{'id': 10036645978, 'score': 0.8213539, 'text'...","{'id': 10001755491, 'score': 0.820079799999999...",Petra van Staveren won the gold medal in the 1...,The wiki page does not contain any information...,The provided wiki page does not contain any in...,The provided Wikipedia page does not contain a...,The provided Wikipedia page does not contain a...
101,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]","[-6.199614048, -13.7564926147, -14.0058326721,...","[The, ĠE, lys, iph, ale, Ġorder, Ġcontains, Ġ,...","{'id': 10005306816, 'score': 0.9049225, 'text'...","{'id': 10056920693, 'score': 0.850604999999999...","{'id': 10057301369, 'score': 0.85027695, 'text...","{'id': 10000059916, 'score': 0.8494835, 'text'...","{'id': 10074864674, 'score': 0.8475151, 'text'...",The Erysiphales order contains 28 genera.,The wiki page provided does not contain inform...,The provided wiki page does not contain inform...,The model output is incorrect based on the inf...,The wiki page provided does not contain inform...
102,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6000000000000001, 'end...","[[0, 3], [9, 18], [63, 70], [78, 87]]","[-4.8190689087, -16.5279369354, -10.1344690323...","[Yes, ,, Ġall, Ġar, ach, n, ids, Ġhave, Ġanten...","{'id': 10000187315, 'score': 0.87926865, 'text...","{'id': 10073653770, 'score': 0.85497665, 'text...","{'id': 10019827221, 'score': 0.8449993, 'text'...","{'id': 10002925248, 'score': 0.84150314, 'text...","{'id': 10047504224, 'score': 0.838115699999999...",The model output is incorrect regarding arachn...,The model output is incorrect based on the pro...,The model output is incorrect. According to th...,The model output is incorrect. According to th...,The model output is incorrect. According to th...
103,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]","[-6.2555613518, -10.9392805099, -13.1335763931...","[Ch, ance, Ġthe, Ġrapper, Ġdebuted, Ġin, Ġ, 20...","{'id': 10038736534, 'score': 0.8665199, 'text'...","{'id': 10044176504, 'score': 0.858650199999999...","{'id': 10020606921, 'score': 0.839050299999999...","{'id': 10043723672, 'score': 0.8334007, 'text'...","{'id': 10009906658, 'score': 0.82527924, 'text...","Chance the Rapper, born Chancelor Johnathan Be...","Chance the Rapper released his debut mixtape, ...",The provided wiki page does not contain inform...,"Chance the Rapper's debut mixtape, ""10 Day,"" w...",The wiki page does not provide a specific debu...
104,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]","[-3.9737114906, -12.9197320938, -7.2846975327,...","[The, ĠUN, ', s, ĠSustainable, ĠCity, Ġinitiat...","{'id': 10005284746, 'score': 0.90800095, 'text...","{'id': 10064524848, 'score': 0.8995247, 'text'...","{'id': 10001721470, 'score': 0.856357599999999...","{'id': 10003102824, 'score': 0.85105896, 'text...","{'id': 10013275110, 'score': 0.850458

In [14]:
df.iloc[0][f'doc1']['text']

'Petronella ("Petra") Grietje van Staveren (born 2 June 1966) is a former breaststroke swimmer from the Netherlands who won the gold medal in the 100 meter breaststroke at the 1984 Summer Olympics in Los Angeles. She also won a bronze at the 1986 world championships and a European silver in 1983 in the 4×100 meter medley relay. She finished five times in fourth place at European championships in 1981–1985.\n\nReferences\n\n1966 births\nLiving people\nOlympic swimmers for the Netherlands\nSwimmers at the 1984 Summer Olympics\nOlympic gold medalists for the Netherlands\nPeople from Kampen, Overijssel\nDutch female breaststroke swimmers\nWorld Aquatics Championships medalists in swimming\nEuropean Aquatics Championships medalists in swimming\nKnights of the Order of Orange-Nassau\n\nMedalists at the 1984 Summer Olympics\nOlympic gold medalists in swimming\nSportspeople from Overijssel'

In [11]:
for i in range(1, 3+1):
    print(df.iloc[0][f'doc_{i}'])

Petra van Staveren won the gold medal in the 100 meter breaststroke at the 1984 Summer Olympics in Los Angeles.
The wiki page does not contain any information about Petra van Stoveren winning a silver medal in the 2008 Summer Olympics. The page is about Petra de Boer-Grimbergen, a Dutch cyclist and speed skater, who competed in the 1992 Summer Olympics and did not win any medals.
The provided wiki page does not contain any information about Petra van Stoveren or her Olympic achievements. The page is about the city of Stavoren in the Netherlands. Therefore, there are no key facts from the wiki page to verify the model output.


In [6]:
features = [f'doc_{i}' for i in range(1, 6)] + ['id', 'model_input', 'model_output_text']

df[features].to_json('/home/admin/projects/semeval2025/rag/en_val_rag_summary.jsonl', lines=True, orient='records')

In [16]:
df.iloc[0]['doc1']['text']

'Petronella ("Petra") Grietje van Staveren (born 2 June 1966) is a former breaststroke swimmer from the Netherlands who won the gold medal in the 100 meter breaststroke at the 1984 Summer Olympics in Los Angeles. She also won a bronze at the 1986 world championships and a European silver in 1983 in the 4×100 meter medley relay. She finished five times in fourth place at European championships in 1981–1985.\n\nReferences\n\n1966 births\nLiving people\nOlympic swimmers for the Netherlands\nSwimmers at the 1984 Summer Olympics\nOlympic gold medalists for the Netherlands\nPeople from Kampen, Overijssel\nDutch female breaststroke swimmers\nWorld Aquatics Championships medalists in swimming\nEuropean Aquatics Championships medalists in swimming\nKnights of the Order of Orange-Nassau\n\nMedalists at the 1984 Summer Olympics\nOlympic gold medalists in swimming\nSportspeople from Overijssel'

In [21]:
txt = df.iloc[0]['doc1']['text']
model_output = df.iloc[0]['model_output_text']
model_input = df.iloc[0]['model_input']
prompt = f'You are data analysis assistant, your task is to summarise wiki page according to model output. You will be supplied with model output and wiki, your task is to write key fact from wiki to verify model output. Write only facts from wiki page.\nmodel output:\n{model_output}\nmodel input:\n{model_input}\nwiki page:\n{txt}'
response = send_qwen(prompt)

In [25]:
model_input

'What did Petra van Staveren win a gold medal for?'

In [26]:
model_output

'Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China.'

In [27]:
response

'The model output is incorrect. According to the wiki page, Petra van Staveren won a gold medal in the 100 meter breaststroke at the 1984 Summer Olympics in Los Angeles, not a silver medal in the 2008 Summer Olympics in Beijing.'

In [49]:
LANGS = ['en', 'es', 'fi', 'fr', 'hi', 'it', 'sv', 'zh']

In [56]:
for f in [f'/home/admin/projects/semeval2025/baseline/participant_kit/mushroom.{lang}-val.v2.jsonl' for lang in LANGS]:
    df = pd.read_json(f, lines=True)
    df = df[['id', 'lang', 'model_input', 'model_output_text', 'model_id', 'soft_labels', 'hard_labels', 'model_output_tokens', 'model_output_logits']]
    df.to_json(f, orient='records', lines=True)

In [57]:
df.columns

Index(['id', 'lang', 'model_input', 'model_output_text', 'model_id',
       'soft_labels', 'hard_labels', 'model_output_tokens',
       'model_output_logits'],
      dtype='object')

In [53]:
['id', 'lang', 'model_input', 'model_output_text', 'model_id', 'soft_labels', 'hard_labels', 'model_output_logits', 'model_output_tokens']
['id', 'lang', 'model_input', 'model_output_text', 'model_id', 'soft_labels', 'hard_labels', 'model_output_tokens', 'model_output_logits']

['hard_labels',
 'id',
 'lang',
 'model_id',
 'model_input',
 'model_output_logits',
 'model_output_text',
 'model_output_tokens',
 'soft_labels']

In [14]:
df['doc_1'][1]

'Title:\nGallia (disambiguation)\nContent:\nGallia (English: Gaul), was a region of Western Europe during the Iron Age occupied by present-day France, Belgium and other neighbouring countries.\n\nGallia or Gallian may also refer to:\n\nPlaces\n\nRoman  provincia\nGallia Cisalpina ("Gaul on this side of the Alps"), a propraetorial province sometimes referred to as Gallia Citerior (Hither Gaul), Provincia Ariminum, or Gallia Togata (Toga-wearing Gaul) \nGallia Transpadana, the part of Cisalpine Gaul between the Padus (now Po) and the Alps\nGallia Cispadana, the part of Cisalpine Gaul to the south of the river Padus (Po)\nGallia Narbonensis, also known as Transalpina (Transalpine France), meaning "Gaul on the other side of the Alps"\nGallia Comata (divided in 22 BC)\n\nOther places\nGallia County, Ohio, a county in southern Ohio, United States\nGallia, Ohio, an unincorporated community\nGallia (Γαλλία), the Greek name for France\n\nArts and entertainment\nGallia (novel), an 1895 novel by 

In [11]:
features = [f'doc_{i}' for i in range(1, 6)] + ['id', 'model_input', 'model_output_text']

df[features][df['lang'] == 'EN'].to_json('/home/admin/projects/semeval2025/rag/en_val_rag.jsonl', lines=True, orient='records')

# Write crits

In [19]:
df = pd.read_json('/home/admin/projects/semeval2025/predictions/prompt_0_predictions.json', lines=True)

In [21]:
def calc_iou(gold, pred):
    m = max([e for s, e in gold] + [e for s, e in pred])
    pred_arr = np.zeros(m, dtype=float)
    for s, e in pred:
        pred_arr[s:e] = 1.0
    gold_arr = np.zeros(m, dtype=float)
    for s, e in gold:
        gold_arr[s:e] = 1.0
    
    i = np.sum(pred_arr * gold_arr)
    u = np.sum(pred_arr) + np.sum(gold_arr) - i
    return i / (u + 1e-4)

In [22]:
def calc_len_diff(gold, pred):
    return len(pred) - len(gold)

In [29]:
for i in df.index:
    df.loc[i, 'len_diff'] = calc_len_diff(df.loc[i, 'golden_hard_labels'], df.loc[i, 'hard_labels'])
    df.loc[i, 'iou'] = calc_iou(df.loc[i, 'golden_hard_labels'], df.loc[i, 'hard_labels'])
    df.loc[i, 'len_pred'] = len(df.loc[i, 'hard_labels'])
    df.loc[i, 'len_gold'] = len(df.loc[i, 'golden_hard_labels'])

In [46]:
path = '/home/admin/projects/semeval2025/problematic_samples/v1/dsat.jsonl'
with open(path, 'a') as f:
    df[(df['len_pred']>0)&(df['len_gold']>0)].sort_values(by='iou', ascending=True).head(5).to_json(f, lines=True, orient='records')
    df.sort_values(by='len_diff', ascending=True).head(5).to_json(f, lines=True, orient='records')
    df.sort_values(by='len_diff', ascending=False).head(5).to_json(f, lines=True, orient='records')
    df[(df['len_pred']>0)&(df['len_gold']>0)].sort_values(by='len_diff', ascending=True).head(5).to_json(f, lines=True, orient='records')
    df[(df['len_pred']>0)&(df['len_gold']>0)].sort_values(by='len_diff', ascending=False).head(5).to_json(f, lines=True, orient='records')

# Analysis

In [47]:
dsat = pd.read_json('/home/admin/projects/semeval2025/predictions/prompt_0_predictions.json', lines=True)

In [48]:
dsat

,id,hard_labels,model_input,model_output_text,response,golden_hard_labels,golden_soft_labels
0,val-en-1,"[[25, 31], [45, 49], [69, 83]]",What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,"silver [sep] 2008 [sep] Beijing, China","[[25, 31], [45, 49], [69, 83]]","[{'start': 10, 'prob': 0.2, 'end': 12}, {'star..."
1,val-en-2,"[[30, 31], [4, 14]]",How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,5 [sep] Elysiphale,"[[30, 31]]","[{'start': 4, 'prob': 0.2, 'end': 14}, {'start..."
2,val-en-3,"[[0, 3], [9, 18], [24, 32], [43, 87]]",Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",Yes [sep] arachnids [sep] antennas [sep] not a...,"[[0, 3], [9, 18], [63, 70], [78, 87]]","[{'start': 0, 'prob': 0.6000000000000001, 'end..."
3,val-en-4,"[[29, 33]]",When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,2011,"[[29, 33]]","[{'start': 18, 'prob': 0.0909090909, 'end': 29..."
4,val-en-5,"[[0, 36], [37, 67], [70, 228]]",What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,The UN's Sustainable City initiative [sep] def...,"[[70, 227]]","[{'start': 0, 'prob': 0.0909090909, 'end': 9},..."
5,val-en-6,"[[101, 109], [238, 246], [302, 308], [345, 362]]",What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,Jiangnan [sep] Zhaozhou [sep] Xiamen [sep] Can...,"[[101, 109], [154, 158], [163, 170], [210, 220...","[{'start': 0, 'prob': 0.2727272727, 'end': 7},..."
6,val-en-7,"[[10, 18], [53, 65], [87, 95]]","If today is 14th October, and it is not a leap...","There are 365 days in a year. So, if today's d...",365 days [sep] October 13th [sep] 162 days,"[[61, 65], [87, 90]]","[{'start': 0, 'prob': 0.1, 'end': 10}, {'start..."
7,val-en-8,"[[4, 11], [50, 57]]",Which network released the TV series of the Th...,The Puncher network that released this TV show...,Puncher [sep] Netflix,"[[4, 11]]","[{'start': 0, 'prob': 0.2727272727, 'end': 4},..."
8,val-en-9,"[[61, 79], [6, 10]]",What is the population of the Spanish region o...,"As of 2021, the estimated population in the re...",1.5 million people [sep] 2021,"[[6, 10], [61, 72]]","[{'start': 0, 'prob': 0.0909090909, 'end': 3},..."
9,val-en-10,"[[20, 24], [108, 116]]",How many stages of labour are there in childbi...,There are typically four stages in labour: the...,four [sep] delivery,"[[43, 116]]","[{'start': 0, 'prob': 0.0909090909, 'end': 43}..."
